In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from learntodrive.utils import move_target_to_cuda
from learntodrive.utils import move_data_to_cuda
from learntodrive.utils import log_textfile
from learntodrive.utils import loadPickle
from learntodrive.utils import get_features_3

from learntodrive.submission import predict_submission
from learntodrive.submission import create_submission

from learntodrive.dataloader_buckets import Drive360Loader

from learntodrive.validation import run_validation

from learntodrive.models.BiGRU import BiGRU
from learntodrive.models.BiGRU import BiGRU_DO


import torch
import torch.nn as nn
import torch.optim as optim
import time
from random import shuffle
import numpy as np
import json
import sys

### Configuration file

In [3]:
config = json.load(open('config_sample1.json'))

In [4]:
config

{'cuda': {'use': True},
 'front': True,
 'multi_camera': {'right_left': False, 'rear': False},
 'data_loader': {'historic': {'number': 10, 'frequency': 1},
  'data_dir': './Data/Sample1/',
  'train': {'csv_name': 'train_sample1.csv',
   'batch_size': 2,
   'shuffle': True,
   'num_workers': 8},
  'validation': {'csv_name': 'val_sample1.csv',
   'batch_size': 2,
   'shuffle': True,
   'num_workers': 8},
  'test': {'csv_name': 'test_sample1.csv',
   'batch_size': 2,
   'shuffle': False,
   'num_workers': 1}},
 'target': {'normalize': True,
  'mean': {'canSteering': -5.406788214535221, 'canSpeed': 13.426163367846936},
  'std': {'canSteering': 73.41232589456718, 'canSpeed': 7.8257638553586455}},
 'image': {'norm': {'mean': [0.4443069311879691,
    0.44355877047930287,
    0.4447293861201888],
   'std': [0.08480363653014882, 0.08435648892210044, 0.08600841133226468]}}}

In [5]:
# config['data_loader']['historic']['frequency'] = 1
# config['data_loader']['historic']['number'] = 10
config['data_loader']['train']['batch_size'] = 65
config['data_loader']['validation']['batch_size'] = 65
config['data_loader']['test']['batch_size'] = 65

densenet121_aug = '../Pickle/sample1_extract_densen121_epoch90_aug_own_features_v3_v2.pickle'
resnet34_high = '../Pickle/sample1_resnet34_high_features_v3_v2.pickle'
densenet201_high = '../Pickle/sample1_densenet201_features_v3_v2.pickle'
dict_pickle = {
    'densenet121_aug': densenet121_aug,
    'resnet34_high': resnet34_high,
    'densenet201_high': densenet201_high
}

### A simple driving model training and evaluation pipeline using the Drive360 dataset and tensorflow.

In [6]:
exp = []

In [7]:
exp.append(
    {
        'modelclass': BiGRU_DO,
        'modelname': 'C_BiGRU_',
        'num_lstm_layers': 3,
        'hidden_lstm_size': 64,
        'p_dropout': 0.6,
        'epochs': 60,
        'loss': 'both',     #either both, steer, speed
        'bins': None,
        'num_bins_min': 2000,
        'sample_0': False,
        'use_classes': False, #true or false
        'lr': 0.003,
        'lr_decay': [20, 30, 40],
        'pickle_files': ['densenet121_aug', 'resnet34_high', 'densenet201_high']   
    }
)

In [8]:
train_loader = None
validation_loader = None
test_loader = None
pickle_files = None

In [9]:
import gc
gc.collect()

44

In [ ]:
for exps in exp:
    # create a train, validation and test data loader
    train_loader = Drive360Loader(config, 'train', exps['bins'], exps['num_bins_min'], exps['sample_0'])
    validation_loader = Drive360Loader(config, 'validation')
    test_loader = Drive360Loader(config, 'test')

    # print the data (keys) available for use. See full 
    # description of each data type in the documents.
    print('Loaded train loader with the following data available as a dict.')
    print(train_loader.drive360.dataframe.keys())
    epochs = exps['epochs']
    modelname = exps['modelname']
    use_classes = exps['use_classes']
    num_lstm_layers = exps['num_lstm_layers']
    hidden_lstm_size = exps['hidden_lstm_size']
    bins = exps['bins']
    p_dropout = exps['p_dropout']
    if bins==None:
        num_classes = 1
    else:
        num_classes = len(bins)-1
    # model path to save in google drive
    MODEL_FINAL_PATH = 'models/'+ modelname + 'model_final.pth'
    STEER_PATH = 'models/'+ modelname + 'model_steer.pth'
    SPEED_PATH = 'models/'+ modelname + 'model_speed.pth'
    LOGFILE_PATH = 'logs/' + modelname + 'logfile.log'
    SUBMISSION_FILENAME = 'Submissions/'+ modelname + 'submission.csv'
    SUBMISSION_FILENAME_FINAL = 'Submissions/'+ modelname + '_finalmodel_submission.csv'
    VALIDATION_FILENAME = 'Validation/'+ modelname + 'validation.csv'
    VAILDATION_FILENAME_FINAL = 'Validation/'+ modelname + '_finalmodel_validation.csv'

    pickle_files = [loadPickle(dict_pickle[x]) for x in exps['pickle_files']]
    num_hidden_features = 0
    for x in pickle_files:
        num_hidden_features += x[list(x.keys())[0]].shape[0]

    modelclass = exps['modelclass']
    model = modelclass(num_cnn_features=num_hidden_features, 
                       num_lstm_layers=num_lstm_layers, 
                       hidden_lstm_size=hidden_lstm_size, 
                       num_classes=num_classes,
                       p_dropout=p_dropout).cuda()
    criterion_speed = nn.MSELoss()
    if exps['use_classes']:
        criterion_angle = nn.CrossEntropyLoss()
    else:
        criterion_angle = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=exps['lr'])
    current_lr = exps['lr']
    lr_decay_idx = 0
    lrates = [0.003, 0.003, 0.001, 0.001]
    lowestSpeedLoss = 999999
    lowestSteerLoss = 999999
    log_textfile(LOGFILE_PATH, 'start')
    log_textfile(LOGFILE_PATH, str(exps))
    for epoch in range(epochs):
        total_loss = 0.0
        running_loss = 0.0
        running_model = 0.0
        running_load = 0.0
        end = time.time()
        model.train()
        for batch_idx, (data, target, front_name) in enumerate(train_loader):
            data = move_data_to_cuda(data)
            target = move_target_to_cuda(target)
            hidden_features = [get_features_3(front_name, x).cuda() for x in pickle_files]
            start = time.time()
            delta_load = start-end
            running_load += delta_load
            optimizer.zero_grad()
            prediction = model(data, hidden_features)
            loss_speed = criterion_speed(prediction['canSpeed'], target['canSpeed'])
            if use_classes:
                target['canSteering'] = target['canSteering'].long().cuda()
            loss_steering = criterion_angle(prediction['canSteering'], target['canSteering'])
            if exps['loss'] == 'steer':
                loss = loss_steering
            if exps['loss'] == 'speed':
                loss = loss_speed
            if exps['loss'] == 'both':
                loss = loss_speed+loss_steering
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            total_loss += loss.item()
            end = time.time()
            delta_model = end - start
            running_model += delta_model
            if (batch_idx+1) % 20 == 0:  
                log_textfile(LOGFILE_PATH, '[epoch: %d, batch:  %5d] loss: %.5f time load: %.5f time model: %.5f' % (epoch + 1, batch_idx + 1, running_loss / 20.0, running_load / 20.0, running_model / 20.0))
                running_loss = 0.0
                running_model = 0.0
                running_load = 0.0
        if epoch+1 in exps['lr_decay']:
            current_lr = current_lr / 2
            for param_group in optimizer.param_groups:
                param_group["lr"] = current_lr
            log_textfile(LOGFILE_PATH, 'LR:' + str(current_lr))
            lr_decay_idx += 1
        shuffle(train_loader.drive360.indices)
        if use_classes:
            mse_steer, mse_speed = run_validation(model, validation_loader, config, pickle_files, train_loader=train_loader)
        else:
            mse_steer, mse_speed = run_validation(model, validation_loader, config, pickle_files, train_loader=None)
        torch.save(model, MODEL_FINAL_PATH)
        if mse_steer < lowestSteerLoss:
            torch.save(model, STEER_PATH)
            lowestSteerLoss = mse_steer
        if mse_speed < lowestSpeedLoss:
            torch.save(model, SPEED_PATH)
            lowestSpeedLoss = mse_speed
        log_textfile(LOGFILE_PATH, "Epoch: " + str(epoch + 1) + "/" + str(epochs) + " Tr loss: " + str(round(total_loss/batch_idx,3)) + " MSESteer: " + str(round(mse_steer,3)) + " MSESpeed: " + str(round(mse_speed,3)) + " Lowest MSESteer: " + str(round(lowestSteerLoss,3)) + " Lowest MSESpeed: " + str(round(lowestSpeedLoss,3)))
    print('Best Steer Loss:', lowestSteerLoss)
    print('Best Speed Loss:', lowestSpeedLoss)



Phase: train # of data: 156029
Phase: validation # of data: 10266
Phase: test # of data: 27920
Loaded train loader with the following data available as a dict.
Index(['cameraRight', 'cameraFront', 'cameraRear', 'cameraLeft', 'canSteering',
       'canSpeed', 'chapter'],
      dtype='object')
start
{'modelclass': <class 'learntodrive.models.New_BasicBiGRU_BatchNorm_BigTopLayer_DenseNet_Buckets.New_BasicBiGRUBN_BigDenseNet_Buckets_DO'>, 'modelname': 'C_BiGRU_', 'num_lstm_layers': 3, 'hidden_lstm_size': 64, 'p_dropout': 0.6, 'epochs': 60, 'loss': 'both', 'bins': None, 'num_bins_min': 2000, 'sample_0': False, 'use_classes': False, 'lr': 0.003, 'lr_decay': [20, 30, 40], 'pickle_files': ['densenet121_aug', 'resnet34_high', 'densenet201_high']}
[epoch: 1, batch:     20] loss: 1.34074 time load: 0.15670 time model: 0.05113
[epoch: 1, batch:     40] loss: 0.78753 time load: 0.09134 time model: 0.04754
[epoch: 1, batch:     60] loss: 0.59022 time load: 0.07860 time model: 0.04733
[epoch: 1, batc

In [15]:
modelSteer = torch.load(STEER_PATH)
modelSpeed = torch.load(SPEED_PATH)
if use_classes:
    df = predict_submission(modelSteer, modelSpeed, pickle_files, test_loader, config, train_loader=train_loader)
else:
    df = predict_submission(modelSteer, modelSpeed, pickle_files, test_loader, config, train_loader=None)
create_submission(df, '../Data/test_full.csv', SUBMISSION_FILENAME)

modelSteer = torch.load(MODEL_FINAL_PATH)
modelSpeed = torch.load(MODEL_FINAL_PATH)
if use_classes:
    df = predict_submission(modelSteer, modelSpeed, pickle_files, test_loader, config, train_loader=train_loader)
else:
    df = predict_submission(modelSteer, modelSpeed, pickle_files, test_loader, config, train_loader=None)
create_submission(df, '../Data/test_full.csv', SUBMISSION_FILENAME_FINAL)

Submission file has no NAs!
Submission file has no NAs!
